# 1. Анализ текстов с использованием метода Word2vec

Word2vec - это еще один агоритм преобразования текстов в точки в векторном пространстве признаков. В его основе лежит нейронная сеть - автоэнкодер, который преобразует слово в вектор фиксированной длины. Особенность этого алгоритма заключается в том, что он учитывает семантику слов. Близкие по смыслу слова будут располагаться ближе друг к другу в векторном пространстве, а далекие - дальше. Если объем текстов достаточно большой, то с помощью модели word2vec мы можем определять синонимы и антонимы слов, и, благодаря этому, точность классификации текстов может увеличиться.

Сегодня будем пользоваться теми же наборами текстов, что и в прошлый раз. Прежде всего установим уже известные вам библиотеки для работы с текстовыми данными.


In [ ]:
!pip install pymorphy2 nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")

Сейчас мы готовы к проведению предобработки текста. Но для того, чтобы работать с моделью word2vec, нам еще понадобится установить библиотеку gensim, в которой описана эта модель.

In [ ]:
!pip install gensim

Для демонстрации возможностей этой библиотеки снова возьмем набор данных с текстами твитов, где сразу же переименуем класс -1 в 0. Вспомним, как выглядит наш датасет.

In [ ]:
from google.colab import drive
import os
# drive.mount('/content/drive/')
drive.mount("/content/drive/", force_remount=True)

In [ ]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/data/tweets_example.xlsx')
df.positive[df.positive==-1] = 0
df.loc[16:25]

Так же, как и при использовании изученных раньше алгоритмов векторизации текста, нам нужно провести очистку данных, т.е. привести все слова текстов к одинаковому виду и форме. Напишем для этого функцию, чтобы можно было потом все эти действия вызывать одной командой.

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2


morph = pymorphy2.MorphAnalyzer()


def text_preprocessing(text):
    """Функция принимает строку и возвращает список слов в начальной форме"""
    text = text.lower()                                                         # приводим текст к нижнему регистру
    text = re.sub(r"[^А-Яа-я]", " ", text)                                      # удаляем все некириллические символы
    words = word_tokenize(text)                                                 # разбиваем тексты на списки слов
    words = [morph.parse(word)[0].normal_form for word in words]                # приводим слова к начальной форме
    words = [word for word in words if word not in stopwords.words("russian")]  # удаляем слова из стоп-листа
    return words

    

In [ ]:
preprocessed_df = df[["text", "positive"]]
preprocessed_df.text = df.text.apply(text_preprocessing)
preprocessed_df[15:25]

Обратите внимание, на этот раз мы не удаляем редкие слова, поскольку теперь текст характеризуется не частотностью тех или иных слов, а семантикой слов, входящих в данный текст.

Теперь обучим модель word2vec. Поскольку это не предиктивная модель, будем обучать ее на всём множестве текстов, чтобы получить векторное представление как можно большего количества слов. Подробно о параметрах модели можно почитать в официальной [документации](https://radimrehurek.com/gensim/models/word2vec.html).

In [ ]:
from gensim.models import Word2Vec

w2v = Word2Vec(size=300, min_count=1)  # создадим экземпляр модели word2vec. Здесь size - размер векторного пространства,
                                       # min_count - минимальное количество появлений слова в наборе данных, при котором
                                       # будем учитывать это слово в модели
w2v.build_vocab(preprocessed_df.text)  # обучим модель на нашем наборе текстов

Теперь посмотрим, какую информацию о словах мы можем получить из обученной модели. Например, для выбранного нами слова можем посмотреть список наиболее похожих на него слов с точки зрения модели.

In [ ]:
w2v.wv.most_similar(positive="ерунда")  # этот метод возвращает список кортежей, где первый элемент - это слово,
                                        # а второй - степень схожести со словом "ерунда". Чем ближе это число к 1,
                                        # тем ближе по смыслу выведенное слово

Аналогично можем посмотреть наименее похожие слова:

In [ ]:
w2v.wv.most_similar(negative=["ерунда"])  # аргумент negative говорит о том, что нужно искать наименее похожие слова.
                                          # В этом случае числа - это степень непохожести.
                                          # Чем ближе к 1, тем слово меньше похоже на "ерунду"

Метод `most_similar` может помочь нам определить слова, которые наиболее похожи на один набор слов и наименее похожи на другой набор слов.

In [ ]:
w2v.wv.most_similar(positive=["подняться", "угата"], negative=["погибать", "клоун"])

# на этот раз цифры - это некая общая метрика похожести на то, что мы просим

Еще можно посмотреть, насколько похожи два слова из выборки

In [ ]:
w2v.wv.similarity("клоун", "работа")  # ответ может быть отрицательным - это будет означать, что
                                         # эта пара слов - больше антонимы, чем синонимы

Очевидно, что с теми словами, которых не было в обучающей выборке, модель работать не сможет:

In [ ]:
w2v.wv.most_similar("сбербанк")

А теперь давайте посмотрим на график, какие слова как расположены друг относительно друга. По умолчанию модель word2vec отображает все слова в пространство размерности 300. Это означает, что каждое слово превращается в набор из 300 чисел. На мониторе такую размерность отобразить очень сложно, поэтому воспользуемся методом снижения размерности векторного пространства t-SNE. Сожмем наши вектора до размерности 2, чтобы их легко можно было отобразить на плоскости.

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (15, 10)


def reduce_dimensions(w2v_model):
    """Фукнция принимает модель word2vec и возвращает массив абсцисс,
    массив ординат и массив слов после снижения размерности"""
    tsne = TSNE(n_components=2, random_state=256)  # создадим экземпляр модели TSNE
    vectors = np.asarray(w2v_model.wv.vectors)     # возьмем из модели 300-мерный массив слов-векторов
    labels = np.asarray(w2v_model.wv.index2word)   # отдельно сохраним соответствие номера вектора и самого слова
    vectors = tsne.fit_transform(vectors)          # проведем преобразование каждого вектора в 2-мерный

    x = [v[0] for v in vectors]                    # запишем отдельно массив абсцисс и массив ординат
    y = [v[1] for v in vectors]
    return x, y, labels


def plot_w2v(w2v_model):
    """Функция строит график распределения слов по векторному пространству
    размерности 2 исходя из обученной модели word2vec"""
    x, y, labels = reduce_dimensions(w2v_model)                      # получим значения по осям и названия точек (исходные слова)
    plt.scatter(x, y)                                                # строим график с точками
    words_to_show_indices = np.random.randint(len(labels), size=25)  # выберем 25 случайных слов, которые отобразим на графике
    for i in words_to_show_indices:
        plt.annotate(labels[i], (x[i], y[i]))                        # для каждого из этих 25 слов отобразим текст на картинке


plot_w2v(w2v)                                                        # применим написанные функции к обученной модели

Кроме вышеперечисленных возможностей, можно обучить модель word2vec на предсказание следующих слов. Обучим ее и попробуем предсказать продолжение твита "Котёнка вчера носик разбила, плакала и расстраивалась :("

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=10000)
w2v.predict_output_word(["вчера", "носик", "разбить"])

Вообще говоря, списки синонимов и антонимов у нас получились достаточно спорные. Причина этого - малая выборка слов, на которых обучалась модель. Внутри word2vec используется нейросеть-автоэнкодер, а таким моделям всегда нужно много данных для того, чтобы составить корректное отображение входных данных в векторное пространство. Чем больше слов в тексте, тем понятней, какие слова похожи по значению, а какие наоборот противоположны.

# Задание 1
1. Проведите предобработку текстов из файлов positive.csv, negative.csv. Нужно выполнить те же действия, что в предыдущем дне, но не удалять редко встречающиеся слова. Регулировать использование редких слов будем на уровне модели word2vec. Не забудьте удалить стоп-слова.
2. Будем исследовать то, как влияют на качество преобразования *размер целевого векторного пространства* и *использование редких слов*. Создайте несколько моделей word2vec, перебрав параметры:
  - размер результирующего пространства: [10, 300, 500] при фиксированной минимальной встречаемости слов = 10
  - минимальная встречаемость слов: [1, 10, 100] при фиксированном размере результирующего векторного пространства = 300

  Обучите их на всем пространстве текстов.

3. Отберите 5 случайных слов из выборки позитивных публикаций и 5 случайных слови из выборки негативных публикаций.
4. Для каждой из обученных моделей найдите по 15 синонимов и по 15 антонимов для каждого из слов из п.3. Опишите:
  - как влияет размер результирующего пространства на точность определения синонимов/антонимов моделью? почему?
  - как влияет минимальная встречаемость слов на точность определения синонимов/антонимов моделью? почему?
5. Постройте графики распределения слов в двумерном пространстве. Опишите, как влияют исследуемые параметры на кучность и расположение точек на графике. Почему?
6. Возьмите любой твит, обучите модель word2vec с параметрами по умолчанию и попробуйте предсказать продолжение твита. Также попробуйте предсказать продолжение случайной фразы. Сравните результаты, полученные после обучения моделей с разным количеством эпох обучения.

In [ ]:
!pip install pymorphy2 nltk
import nltk
nltk.download("punkt")
nltk.download("stopwords")

!pip install gensim

from google.colab import drive
import os
drive.mount("/content/drive/", force_remount=True)

In [ ]:
import pandas as pd

columns = ['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'lisy_count']
df_positive = pd.read_csv('/content/drive/MyDrive/school21/positive.csv', sep = ";", names = columns)
df_negative = pd.read_csv('/content/drive/MyDrive/school21/negative.csv', sep = ";", names = columns)
df = pd.concat((df_positive, df_negative), axis=0)
df.positive[df.positive == -1] = 0
df.index = list(range(226834))

In [ ]:
df.text = df.text.str.lower()
df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text.loc[19:22]

In [ ]:
df.to_excel (r'\Users\ebalgruu\Desktop\clean_tweets_df.xlsx', index = False, header=True)
df.to_csv('clean_dataframe.csv', sep='\t', encoding='utf-8')
df.to_pickle('/content/drive/MyDrive/data/clean_tweets_df.pkl')

# df = pd.read_pickle('/content/drive/MyDrive/data/clean_tweets_df.pkl')



---



размер результирующего пространства: [10, 300, 500] при фиксированной минимальной встречаемости слов = 10

минимальная встречаемость слов: [1, 10, 100] при фиксированном размере результирующего векторного пространства = 300



In [ ]:
# Токенизация текста и удаление стоп-слов:

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df.text = list(map(word_tokenize, df.text))
russian_stopwords = stopwords.words("russian")
russian_stopwords.sort()
russian_stopwords

def delete_stopword(words):
    global russian_stopwords
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
df.text.loc[19:22]

In [ ]:
# Лемматизация:

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))
df.text.loc[19:22]

In [ ]:
preprocessed_df = df[["text", "positive"]]
preprocessed_df

In [ ]:
from gensim.models import Word2Vec

w2v_10_10 = Word2Vec(size=10, min_count=10)
w2v_10_10.build_vocab(preprocessed_df.text)  # создадим словарь
w2v_10_10.train(preprocessed_df.text, total_examples=w2v_10_10.corpus_count, epochs=100) # обучим модель на нашем наборе текстов
w2v_10_10.save('w2v_10_10')

# w2v_300_10 = Word2Vec(size=300, min_count=10) 
# w2v_300_10.build_vocab(preprocessed_df.text)
# w2v_300_10.train(preprocessed_df.text, total_examples=w2v_300_10.corpus_count, epochs=100)
# w2v_300_10.save('w2v_300_10')

w2v_500_10 = Word2Vec(size=500, min_count=10) 
w2v_500_10.build_vocab(preprocessed_df.text)
w2v_500_10.train(preprocessed_df.text, total_examples=w2v_500_10.corpus_count, epochs=100)
w2v_500_10.save('w2v_500_10')

w2v_300_1 = Word2Vec(size=300, min_count=1) 
w2v_300_1.build_vocab(preprocessed_df.text)
w2v_300_1.train(preprocessed_df.text, total_examples=w2v_300_1.corpus_count, epochs=100)
w2v_300_1.save('w2v_300_1')

w2v_300_100 = Word2Vec(size=300, min_count=100) 
w2v_300_100.build_vocab(preprocessed_df.text)
w2v_300_100.train(preprocessed_df.text, total_examples=w2v_300_100.corpus_count, epochs=100)
w2v_300_100.save('w2v_300_100')

w2v_10_10_n = Word2Vec.load('w2v_10_10')
# w2v_300_10_n = Word2Vec.load('w2v_300_10')
w2v_500_10_n = Word2Vec.load('w2v_500_10')
w2v_300_1_n = Word2Vec.load('w2v_300_1')
w2v_300_100_n = Word2Vec.load('w2v_300_100')

In [ ]:
w2v_300_10 = Word2Vec(size=300, min_count=10) 
w2v_300_10.build_vocab(preprocessed_df.text)
w2v_300_10.train(preprocessed_df.text, total_examples=w2v_300_10.corpus_count, epochs=100)
w2v_300_10.save('w2v_300_10')

w2v_300_10_n = Word2Vec.load('w2v_300_10')

In [ ]:
# Случайные слова из датафрейма
import random

set = []

# 5 позитивных слов
for i in range(0, 5):
  str_p = random.choice(list(preprocessed_df[df.positive == 1].text))
  word = random.sample(str_p, 1)
  set.append(word)

# 5 негативных слов
for i in range(0, 5):
  str_p = random.choice(list(preprocessed_df[df.positive == 0].text))
  word = random.sample(str_p, 1)
  set.append(word)

set

In [ ]:
# Синонимы для модели w2v_300_1:

for i in range(0, 10):
  w2v_300_1.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_300_1.wv.most_similar(positive=set[i], topn=15))

# Антонимы для модели w2v_300_1:

for i in range(0, 10):
  w2v_300_1.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_300_1.wv.most_similar(negative=set[i], topn=15))

In [ ]:
# Синонимы для модели w2v_300_10:

for i in range(0, 10):
  w2v_300_10.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_300_10.wv.most_similar(positive=set10[i], topn=15))

# Антонимы для модели w2v_300_10:

for i in range(0, 10):
  w2v_300_10.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_300_10.wv.most_similar(negative=set10[i], topn=15))

In [ ]:
set100 = [['сейчас'],
 ['папа'],
 ['забавный'],
 ['против'],
 ['нужный'],
 ['мой'],
 ['никто'],
 ['наушник'],
 ['есть'],
 ['быть']]

In [ ]:
# Синонимы для модели w2v_300_100:

for i in range(0, 10):
  w2v_300_100.wv.most_similar(positive=set100[i], topn=15)
  #print(w2v_300_100.wv.most_similar(positive=set100[i], topn=15))

# Антонимы для модели w2v_300_100:

for i in range(0, 10):
  w2v_300_100.wv.most_similar(negative=set100[i], topn=15)
  #print(w2v_300_100.wv.most_similar(negative=set100[i], topn=15))

In [ ]:
# Синонимы для модели w2v_10_10:

for i in range(0, 10):
  w2v_10_10.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_10_10.wv.most_similar(positive=set[i], topn=15))

# Антонимы для модели w2v_10_10:

for i in range(0, 10):
  w2v_10_10.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_10_10.wv.most_similar(negative=set[i], topn=15))

In [ ]:
# Синонимы для модели w2v_500_10:

for i in range(0, 10):
  w2v_500_10.wv.most_similar(positive=set[i], topn=15)
  #print(w2v_500_10.wv.most_similar(positive=set10[i], topn=15))

# Антонимы для модели w2v_500_10:

for i in range(0, 10):
  w2v_500_10.wv.most_similar(negative=set[i], topn=15)
  #print(w2v_500_10.wv.most_similar(negative=set10[i], topn=15))

Вывод.
Чем больше размер результирующего пространства, тем выше точность определения синонимов/антонимов моделью потому, что у модели больше критериев оценки близости слов и можно это сделать точнее.
Минимальная встречаемость слов влияет на точность определения синонимов/антонимов моделью так: чем больше минимальная встречаемость слов, тем больше точность определения синонимов/антонимов. Тем не менее, в результат могут не попасть слова, являющиеся наиболее точными синонимами/антонимамим, но не попавшие в выборку должное количество раз.

In [ ]:
from sklearn.manifold import TSNE
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (15, 10)


def reduce_dimensions(w2v_model):
    """Фукнция принимает модель word2vec и возвращает массив абсцисс,
    массив ординат и массив слов после снижения размерности"""
    tsne = TSNE(n_components=2, random_state=256)  # создадим экземпляр модели TSNE
    vectors = np.asarray(w2v_model.wv.vectors)     # возьмем из модели 300-мерный массив слов-векторов
    labels = np.asarray(w2v_model.wv.index2word)   # отдельно сохраним соответствие номера вектора и самого слова
    vectors = tsne.fit_transform(vectors)          # проведем преобразование каждого вектора в 2-мерный

    x = [v[0] for v in vectors]                    # запишем отдельно массив абсцисс и массив ординат
    y = [v[1] for v in vectors]
    return x, y, labels


def plot_w2v(w2v_model):
    """Функция строит график распределения слов по векторному пространству
    размерности 2 исходя из обученной модели word2vec"""
    x, y, labels = reduce_dimensions(w2v_model)                      # получим значения по осям и названия точек (исходные слова)
    plt.scatter(x, y)                                                # строим график с точками
    words_to_show_indices = np.random.randint(len(labels), size=25)  # выберем 25 случайных слов, которые отобразим на графике
    for i in words_to_show_indices:
        plt.annotate(labels[i], (x[i], y[i]))                        # для каждого из этих 25 слов отобразим текст на картинке
    plt.show()

plot_w2v(w2v_10_10)

In [ ]:
plot_w2v(w2v_300_10)

In [ ]:
plot_w2v(w2v_500_10)

In [ ]:
plot_w2v(w2v_300_1)

In [ ]:
plot_w2v(w2v_300_100)

Вывод.
Чем больше минимальная встречаемость слов, тем реже расположены точки на графике. Это происходит в первую очередь потому, что в рассчет берется меньшее количество слов.
Чем больше размер результирующего пространства, тем более явно наблюдается уплотнение точек ближе к центру. Это связано с тем, что с увеличением размера результирующего пространства повышается точность определения связей между словами. Большой размер результирующего пространства может вызывать переобученность.

Обучим модель по умолчанию и попробуем предсказать продолжение случайной фразы:

---



In [ ]:
from gensim.models import Word2Vec

w2v = Word2Vec(size=300, min_count=2) 
w2v.build_vocab(preprocessed_df.text)

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=100)
w2v.predict_output_word(["такси", "везти", "работа"])

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=10)
w2v.predict_output_word(["такси", "везти", "работа"])

Попробуем предсказать продолжение твита "Котёнка вчера носик разбила, плакала и расстраивалась :("

---



In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=10)
w2v.predict_output_word(["вчера", "носик", "разбить"])

In [ ]:
w2v.train(preprocessed_df.text, total_examples=preprocessed_df.shape[0], epochs=100)
w2v.predict_output_word(["вчера", "носик", "разбить"])

Чем больше эпох обучения, тем более точное предсказание дает модель.

---



# 2. Использование градиентного бустинга над решающими деревьями для решения задачи классификации текстов

В предыдущем разделе мы научились превращать слова в векторы. Но перед нами стоит задача классификации текста, а не одного слова, поэтому нам нужно придумать способ, как целый текст представить числами.

После преобразования отдельные слова стали векторами, значения которых зависят от семантики слова. Будем рассматривать твит как сущность с усредненной семантикой всех содержащихся в нём слов. Таким образом, для преобразования целого текста в вектор, нам нужно получить средний вектор всех содержащихся в нём слов. Такой способ реализован в модели Doc2Vec в библиотеке gensim.

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tweets = [TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_df.text)]  # преобразуем наши тексты в объекты, понятные док-2-веку
d2v = Doc2Vec(tweets, min_count=2)                        # создадим модель Doc2Vec
d2v.train(tweets, total_examples=len(tweets), epochs=20)  # подберем веса коэффициентов внутри модели, которые больше будут подходить к нашему набору текстов

In [ ]:
from sklearn.model_selection import train_test_split

# разобьем набор текстов на тренировочную и тестовую выборки
X_train_texts, X_test_texts, y_train, y_test = train_test_split(preprocessed_df.text, preprocessed_df.positive, test_size=0.2, random_state=21)

In [ ]:
X_train_texts  # пока что наши тексты выглядят как списки слов в начальной форме, но нам нужно получить из этого векторы

In [ ]:
def transform_text_array_to_vector_dataframe(text_array):
    """Функция, которая преобразует одномерный колонку списков слов из текстов
    в датафрейм со значениями векторов этих текстов"""
    columns = [str(n) for n in range(d2v.vector_size)]               # задаем список названий колонок - просто порядковые номера
    vectors_ndarray = text_array.apply(d2v.infer_vector).to_list()  # прогоняем каждый текст через модель doc2vec и формируем многомерный массив чисел
    return pd.DataFrame(vectors_ndarray, columns=columns)            # оборачиваем его в датафрейм для удобства


X_train = transform_text_array_to_vector_dataframe(X_train_texts)    # наконец создадим датафреймы, которые сможем подать в модель классификации
X_test = transform_text_array_to_vector_dataframe(X_test_texts)

Вы уже познакомились с некоторыми "деревянными" методами машинного обучения - решающим деревом и случайным лесом. Градиентный бустинг - это итеративный способ построения классификации, полученный (как и случайный лес) путем комбинации нескольких алгоритмов. Сначала строится обычное решающее дерево. Затем строится ряд моделей, предсказывающих ошибку исходной модели. Эти предсказания вычитаются из исходной модели. Таким образом, в итоге мы имеем один классификатор, но намного более точный, чем обычное решающее дерево. Чем больше итераций этого алгоритма будет проведено, тем выше получится качество модели, но она будет дольше обучаться.

Градиентный бустинг над решающими деревьями - это, пожалуй, самая распространенная на сегодняшний день модель машинного обучения. Ее используют для решениях многих задач, начиная кредитным скорингом и заканчивая товарным спросом и антифродом. Мы будем использовать эту модель для классификации наших текстов.

Модель градиентного бустинга есть в библиотеке sklearn, но на больших данных она будет обучаться долго. На рынке сейчас популярны несколько оптимизированных реализаций градиентного бустинга. Самые известные - xgboost, lightgbm и catboost. Рассмотрим xgboost, но вы можете использовать любую из этих трех.

In [ ]:
# установим библиотеку
!pip install xgboost

Работать с моделью xgboost можно так же, как с моделями sklearn: fit и predict. Основные гиперпараметры - максимальная глубина деревьев модели и количество деревьев.

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

# Задание 2

В этом задании от вас требуется провести классификацию текстов с использованием градиентного бустинга. Постройте такую модель, которая даст наилучший результат по метрике precision к классу 0, подобрав гиперпараметры:
- минимальная встречаемость слова в текстах в doc2vec
- максимальная глубина деревеьев в бустинге
- количество деревьев в бустинге

Дайте ответ на вопрос: лучше использовать более глубокие или более мелкие деревья в модели градиентного бустинга?

In [ ]:
# Задания №2

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
!pip install xgboost

tweets = [TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_df.text)]

# Разобьем набор текстов на тренировочную и тестовую выборки
X_train_texts, X_test_texts, y_train, y_test = train_test_split(preprocessed_df.text, preprocessed_df.positive, test_size=0.2, random_state=21)






In [ ]:
# Создадим модель Doc2Vec с минимальной встречаемостью слова 2
d2v = Doc2Vec(tweets, min_count=2)
d2v.train(tweets, total_examples=len(tweets), epochs=20)  # подберем веса коэффициентов внутри модели, которые больше будут подходить к нашему набору текстов

In [ ]:
def transform_text_array_to_vector_dataframe(text_array):
    """Функция, которая преобразует одномерную колонку списков слов из текстов
    в датафрейм со значениями векторов этих текстов"""
    columns = [str(n) for n in range(d2v.vector_size)]               # задаем список названий колонок - просто порядковые номера
    vectors_ndarray = text_array.apply(d2v.infer_vector).to_list()  # прогоняем каждый текст через модель doc2vec и формируем многомерный массив чисел
    return pd.DataFrame(vectors_ndarray, columns=columns)            # оборачиваем его в датафрейм для удобства


X_train = transform_text_array_to_vector_dataframe(X_train_texts)    # наконец создадим датафреймы, которые сможем подать в модель классификации
X_test = transform_text_array_to_vector_dataframe(X_test_texts)

In [ ]:
# Применим модель xgboost. Основные гиперпараметры: max_depth - максимальная глубина деревьев модели и n_estimators - количество деревьев.
# min_count=2, epochs=20

from xgboost import XGBClassifier
from sklearn.metrics import classification_report

xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

In [ ]:
# Минимальная встречаемость 10
# min_count=10, epochs=20

d2v = Doc2Vec(tweets, min_count=10)
d2v.train(tweets, total_examples=len(tweets), epochs=20)

X_train10 = transform_text_array_to_vector_dataframe(X_train_texts)
X_test10 = transform_text_array_to_vector_dataframe(X_test_texts)

xgb.fit(X_train10, y_train)
y_pred = xgb.predict(X_test10)
print(classification_report(y_pred, y_test))

In [ ]:
# Минимальная встречаемость 100
# min_count=100, epochs=20

d2v = Doc2Vec(tweets, min_count=100)
d2v.train(tweets, total_examples=len(tweets), epochs=20)

X_train100 = transform_text_array_to_vector_dataframe(X_train_texts)
X_test100 = transform_text_array_to_vector_dataframe(X_test_texts)

xgb.fit(X_train100, y_train)
y_pred = xgb.predict(X_test100)
print(classification_report(y_pred, y_test))

In [ ]:
# Применим параметры, общие для всех min_count=10, epochs=10

d2v = Doc2Vec(tweets, min_count=10)
d2v.train(tweets, total_examples=len(tweets), epochs=10)

X_train = transform_text_array_to_vector_dataframe(X_train_texts)
X_test = transform_text_array_to_vector_dataframe(X_test_texts)

In [ ]:
xgb = XGBClassifier(max_depth=5, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

In [ ]:
xgb = XGBClassifier(max_depth=10, n_estimators=50)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

In [ ]:
xgb = XGBClassifier(max_depth=10, n_estimators=10)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

In [ ]:
xgb = XGBClassifier(max_depth=10, n_estimators=100)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
print(classification_report(y_pred, y_test))

Вывод.
Лучше использовать более глубокие деревья в модели градиентного бустинга. Увеличение количества деревьев также улучшает результат